In [ ]:
#Bibliotecas que foram instaladas:
#!pip install matplotlib
#!pip install sqlalchemy
#!pip install seaborn
#!pip install ssl
#!pip install pandas
#!pip install pymongo
#!pip install pandas_gbq
#!pip install scikit-learn
#!pip install PyDrive
#!pip install item_based_recommender

## Estudar:
- Gitignore;
- GitHub Action;
- Curso Rápido de Escrita Técnica da Google;
- Métodos Dunder (double underline);


# CÓDIGO COMEÇANDO

## BIBLIOTECAS

In [1]:
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
import pandas as pd
import seaborn as sns
import matplotlib as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

#gauth = GoogleAuth()
#gauth.LocalWebserverAuth()

#drive = GoogleDrive(gauth)

In [2]:
df_o = pd.read_csv('DNC_orders_dataset.csv', index_col = 0)
df_oi = pd.read_csv('DNC_order_items_dataset.csv', index_col = 0)
df_or = pd.read_csv('DNC_order_reviews_dataset.csv', index_col = 0)
df_p = pd.read_csv('DNC_products_dataset.csv', index_col = 0)

In [3]:
df_o.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99441 entries, 0 to 99440
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   order_id      99441 non-null  object
 1   customer_id   99441 non-null  object
 2   order_status  99441 non-null  object
dtypes: object(3)
memory usage: 3.0+ MB


In [4]:
df_oi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   order_id       112650 non-null  object 
 1   order_item_id  112650 non-null  int64  
 2   product_id     112650 non-null  object 
 3   price          112650 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.3+ MB


In [5]:
df_or.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99224 entries, 0 to 99223
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   review_id     99224 non-null  object
 1   order_id      99224 non-null  object
 2   review_score  99224 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 3.0+ MB


In [6]:
df_or['review_score'].unique()

array([4, 5, 1, 3, 2], dtype=int64)

In [7]:
df_p.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32951 entries, 0 to 32950
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_weight_g            32949 non-null  float64
 5   product_length_cm           32949 non-null  float64
 6   product_height_cm           32949 non-null  float64
 7   product_width_cm            32949 non-null  float64
dtypes: float64(6), object(2)
memory usage: 2.3+ MB


In [ ]:
teste = df_p[df_p['product_weight_g'].isnull()]

In [8]:
df_final = df_oi.merge(df_o[['order_id','order_status']], how='left', on='order_id')

In [9]:
df_final = df_final.merge(df_or[['order_id','review_score']], how='left', on='order_id')

In [10]:
df_final = df_final.merge(df_p[['product_id', 'product_category_name']], how='left', on='product_id')

In [11]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113314 entries, 0 to 113313
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   order_id               113314 non-null  object 
 1   order_item_id          113314 non-null  int64  
 2   product_id             113314 non-null  object 
 3   price                  113314 non-null  float64
 4   order_status           113314 non-null  object 
 5   review_score           112372 non-null  float64
 6   product_category_name  111702 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 6.9+ MB


In [55]:
df_final2 = df_final[['order_id','product_id','product_category_name','order_item_id']].groupby('order_id').agg(products=('product_id','nunique'), quantity=('order_item_id', 'sum')).sort_values(by=['products','quantity'],ascending=False).head(3000)
df_final2.head(20)

,products,quantity
order_id,,
ca3625898fbd48669d50701aba51cd5f,8,55
77df84f9195be22a4e9cb72ca9e8b4c2,7,28
7d8f5bfd5aff648220374a2df62e84d5,7,28
ad850e69fce9a512ada84086651a2e7d,7,28
5a3b1c29a49756e75f1ef513383c0c12,6,132
1c11d0f4353b31ac3417fbfa5f0f2a8a,6,28
200f4d883fcc701355e46b8c6035743f,6,21
2455cbeb73fd04b170ca2504662f95ce,6,21
3990f96693d321ac142fff312bf3706a,6,21


In [56]:
df_teste = df_final[['order_id','product_id','order_item_id']].merge(df_final2,how='inner',on='order_id').pivot_table(index='order_id',columns='product_id',values='order_item_id').fillna(0)
df_teste.head(3)

product_id,0011c512eb256aa0dbbb544d8dffcf6e,001b72dfd63e9833e8c02742adf472e3,0042f1a9a7e0edd1400c6cd0fda065f8,005030ef108f58b46b78116f754d8d38,0060b415594c5e1200324ef1a18493c4,007c63ae4b346920756b5adcad8095de,00ae7076313576f94d9107599d79a978,00ba6d766f0b1d7b78a5ce3e1e033263,00d2fa6f2a87633084c8e0400d41a65a,00f42e56821b1779fbf397fe72639054,...,ff7c7524d5f9a8386ab27088ec03c1fb,ff85ff517698c3fe8b200afddda7fb3a,ff9032482857269e8683f0c2f8ba4709,ff95ac47246ef13e48712ea1ff8df0d9,ffb2e8c1ddc7c3e590d2bc4c91de53e1,ffbc83054b3741a8d67fc59d9cf9d42d,ffcfaba393e8ef71937c6e8421bc2868,ffd4bf4306745865e5692f69bd237893,ffe8083298f95571b4a66bfbc1c05524,fff1059cd247279f3726b7696c66e44e
order_id,,,,,,,,,,,,,,,,,,,,,
002f98c0f7efd42638ed6100ca699b42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00337fe25a3780b3424d9ad7c5a4b35e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
005d9a5423d47281ac463a968b3936fb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
def get_recommendations(df, item):
    
    recommendations = df.corrwith(df[item])
    recommendations.dropna(inplace=True)
    recommendations = pd.DataFrame(recommendations, columns=['correlation']).reset_index()
    recommendations = recommendations.sort_values(by='correlation', ascending=False)
    
    return recommendations

In [58]:
recomendacao = get_recommendations(df_teste, 'fbce4c4cb307679d89a3bf3d3bb353b9')
recomendacao.head()

,product_id,correlation
4540,fbce4c4cb307679d89a3bf3d3bb353b9,1.000000
1855,66d8fb4ea4391a1f36cf50152c26b4ca,0.534306
627,23ab7bb9eac81e85ec43ed71064cf7ce,0.421799
3184,b0c89945c034268074f5f80b362bda34,0.273980
4440,f5889057b0c061390ce20f4b7b842ac0,0.266796
